In [2]:
import pandas as pd
import numpy as np
# import hdbscan
import sklearn
from matplotlib import pyplot as plt
from matplotlib import cm

In [3]:
from astropy.io import fits

In [4]:
hdul = fits.open('http://gal-03.sai.msu.ru/~vtoptun/redshift/rcsed_v2.fits')

In [5]:
hdr0 = hdul[0].header

In [6]:
hdr1 = hdul[1].header

In [7]:
data = hdul[1].data

In [8]:
DATA = pd.DataFrame(np.array(data).byteswap().newbyteorder())

In [9]:
DATA.head()

,ra,dec,z_sdss,zerr_sdss,specObjID_sdss,bestObjID_sdss,plate_sdss,mjd_sdss,fiberID_sdss,z_2df,...,z_wigglez,Q_wigglez,SpecFile_wigglez,Z_gama,NQ_gama,SPECID_gama,GroupID,GroupSize,multiple_z,dz
0,0.000019,-4.760800,2.467120,0.002194,7.919679e+18,0,7034,56564,360,NaN,...,NaN,-32768,b' ',NaN,-32768,b' ',-2147483648,-2147483648,70,70
1,0.000067,-26.058128,NaN,NaN,NaN,-9223372036854775808,-32768,-2147483648,-32768,0.11854,...,NaN,-32768,b' ',NaN,-32768,b' ',-2147483648,-2147483648,70,70
2,0.000091,24.902252,0.555936,0.000245,7.333218e+18,1237666309095751941,6513,56543,843,NaN,...,NaN,-32768,b' ',NaN,-32768,b' ',-2147483648,-2147483648,70,70
3,0.000346,-6.491940,0.514177,0.000130,8.047035e+18,1237680243597902184,7147,56574,831,NaN,...,NaN,-32768,b' ',NaN,-32768,b' ',-2147483648,-2147483648,70,70
4,0.000446,-33.323280,NaN,NaN,NaN,-9223372036854775808,-32768,-2147483648,-32768,0.27531,...,NaN,-32768,b' ',NaN,-32768,b' ',-2147483648,-2147483648,70,70


In [10]:
glist = pd.read_csv('result_glist_s.csv')

In [11]:
glist.drop(glist.columns[0], axis='columns', inplace=True)

In [13]:
glist.columns

Index(['iGalID', 'iGrID', 'Name', 'RAJ2000_gal', 'DEJ2000_gal', 'z_gal',
       'logMstar_gal', 'Dist_gal', 'RAJ2000_group', 'DEJ2000_group', 'z_group',
       'logLtot', 'logLobs', 'logMtot', 'logMstar_group', 'NMstar', 'logMdyn',
       'sigma', 'Rad', 'angRad', 'DL', 'Ntot', 'Dist_group', 'e_logLtot',
       'e_logMtot'],
      dtype='object')

In [14]:
# index_list = DATA[DATA['z_sdss'].isna()==False].index

In [15]:
# index_list

In [16]:
import joblib
from joblib import Parallel, delayed

In [ ]:
from time import process_time 

# Start the stopwatch / counter  
t1_start = process_time()

coincedence = []

# table[index, ra, dec]

table_glist = (glist[['RAJ2000_gal', 'DEJ2000_gal']].sort_values(by='RAJ2000_gal').reset_index()).to_numpy()
table_rcsed = (DATA[DATA['z_sdss'].isna()==False][['ra', 'dec']].sort_values(by='ra').reset_index()).to_numpy()

def sieve(table_rcsed, i, table_glist):
#     pair = []#{}
    delta = 3/3600
    
    if i <= int(len(table_glist)/2):
        j=0
        while table_rcsed[j, 1] <= table_glist[i, 1] + delta and j < len(table_rcsed):
            if table_glist[i, 1] - delta <= table_rcsed[j, 1]:
                if table_glist[i, 2] - delta <= table_rcsed[j, 2] <= table_glist[i, 2] + delta:
#                     pair.setdefault(int(table_glist[i,0]),int(table_rcsed[j,0]))
                    pair = [int(table_glist[i,0]),int(table_rcsed[j,0])]
                    table_rcsed = np.delete(table_rcsed, j, 0)
                    return pair
                    break
                else:
                    j += 1
            else:
                j += 1
    else:
        j = len(table_rcsed)-1
        while table_rcsed[j, 1] >= table_glist[i, 1] - delta and j >= 0 :
            if table_glist[i, 1] + delta >= table_rcsed[j, 1]:
                if table_glist[i, 2] - delta <= table_rcsed[j, 2] <= table_glist[i, 2] + delta:
#                     pair.setdefault(int(table_glist[i,0]),int(table_rcsed[j,0]))
                    pair = [int(table_glist[i,0]),int(table_rcsed[j,0])]
                    table_rcsed = np.delete(table_rcsed, j, 0)
                    return pair
                    break
                else:
                    j -= 1
            else:
                j -= 1


coincedence.extend(Parallel(n_jobs=2)(delayed(sieve)(table_rcsed, i, table_glist) for i in range(len(table_glist))))
new_coincedence = [x for x in coincedence if x != None]

with open("new_coincedence_1.txt", "w") as output:
    output.write(str(new_coincedence))   
    
t1_stop = process_time() 
   
print("Elapsed time during the whole program in seconds:", 
                                         t1_stop-t1_start) 

In [ ]:
# new_coincedence